# 0. 데이터 준비

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

train_url = '/kaggle/input/demand-forecasting-kernels-only/train.csv'
test_url = '/kaggle/input/demand-forecasting-kernels-only/test.csv'
submission_url = '/kaggle/input/demand-forecasting-kernels-only/sample_submission.csv'

In [ ]:
## 데이터 불러오기

train = pd.read_csv(train_url, parse_dates = ['date'])
test = pd.read_csv(test_url, index_col = 'id', parse_dates = ['date'])

---

# 1. train 데이터 구조 확인

In [ ]:
train.info()

In [ ]:
train.head()

---

## 1-1) sales 변수(예측변수) 확인

1) sales 변수의 전체 분포는 0에서 약 135까지의 수에 몰려있으며, 135 이후에는 소수의 이상값으로 존재한다는 것을 확인하였습니다.\
2) 이상값을 제외한다면 전체적으로 정규분포에 가까운 형태입니다.

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize = (30, 5))
sns.boxplot(train.sales);

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize = (30, 5))
sns.distplot(train.sales);

In [ ]:
train.sales.describe().round(3)

In [ ]:
train.sales.quantile([0, 0.95, 0.99, 1])

In [ ]:
# 99분위 범위를 초과하는 판매량(Outlier)만 뽑아봤습니다. 9195개 데이터가 있습니다.

train[train.sales >= 135]

### date(날짜) 부분에 많은 영향요소가 있다고 판단하여 date는 따로 뒤에서 다루기로 했습니다.

---

## 1-2) store 변수 
1) 매장의 수는 10곳입니다. \
2) sales와의 관계를 보기 위해 기술통계량을 내고 boxplot도 그려보았습니다. \
3) 그 결과, 2와 8이 대체로 눈에 띄게 높은, 최상위 판매량을 나타내는 특성이 있었고,\
4) 3, 10, 4, 9가 다음의 상위 판매량,\
5) 1이 중위 판매량,\
6) 5, 6, 7이 전반적으로 판매량이 낮은, 하위 판매량을 그립니다.

In [ ]:
desc_store = train.groupby('store').sales.describe().round(3)
store_99p = train.groupby('store').sales.quantile(0.99)

pd.concat([desc_store, store_99p], axis = 1).rename({'sales' : '99%'}, axis = 1).sort_values(by = '50%', ascending = False)

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize = (30, 10))
sns.boxplot(data = train, x = 'store', y = 'sales');

---

## 1-3) item 변수

1) 아이템의 수는 50개입니다. 너무도 양이 많아, 비슷한 판매량을 가진 집단끼리 재구조화할 필요가 있을 것 같았습니다. \
2) 일단 역시 sales와의 관계를 보기 위해 기술통계량을 내고 boxplot도 그려보았습니다. \

    ※ 기술통계량에 따라 그룹핑도 시도해 보았으나 예측이 너무 빗나가는 문제가 있었습니다.

In [ ]:
desc_item = train.groupby('item').sales.describe().round(3)
item_99p = train.groupby('item').sales.quantile(0.99)

desc_item = pd.concat([desc_item, item_99p], axis = 1).rename({'sales' : '99%'}, axis = 1).sort_values(by = '50%', ascending = False)
item_list = list(desc_item.index)

desc_item.head()

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize = (30, 10))
sns.boxplot(data = train, x = 'item', y = 'sales', order = item_list);

---

## 1-4) date 변수
### date컬럼을 확인해본 결과 2013년부터 2017년의 데이터가 있는 사실상의 시계열 데이터임을 인지하였습니다.
### 여기에, test는 2018년 1월부터 3월까지의 데이터만 있습니다.

In [ ]:
train.date

In [ ]:
test.date

### 날짜와 판매량(sales)의 관계에 대해 세 가지 가설을 세웠습니다.

**1) 시간이 지나면서 전반적인 판매량은 상승할 것이다.\
2) 월 또는 매주마다 판매량의 차이가 있을 것이며, 전년 대비 대체로 증가하는 추세를 보일 것이다. \
3) 각 요일마다, 적어도 평일/주말마다 판매량의 차이가 있을 것이다.**

이 점을 주안점으로 생각하고, 날짜 데이터에서 연도, 월, 주, 요일을 각각 분리 추출하기로 하였습니다.

In [ ]:
def date_split(df):
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['week'] = df.date.dt.week
    df['day'] = df.date.dt.day_name()

In [ ]:
train = pd.read_csv(train_url, parse_dates = ['date'])
test = pd.read_csv(test_url, index_col = 'id', parse_dates = ['date'])

date_split(train)
date_split(test)

In [ ]:
train.head()

In [ ]:
# test 데이터도 함께 재구조화합니다.

test.head()

---

# 2. 데이터 재구조화

In [ ]:
## 2-1) 월 변수 분리 (test 데이터의 특징에 맞춰서 train 데이터도 1~3월만 활용)

train = train[train['month'].isin([1, 2, 3])]

In [ ]:
## 2-2) 독립변수(설명변수)/종속변수(예측변수) 분리

train_x = train.drop('sales', axis = 1).copy()
train_y = train['sales'].copy()

test_x = test.copy()

In [ ]:
# ## 2-3) 범주형 변수 인코딩(pd.get_dummies) / store, item, 요일(day로 설정)

train_store_dummies = pd.get_dummies(train.store, prefix = 'store')
test_store_dummies = pd.get_dummies(test.store, prefix = 'store')

train_item_dummies = pd.get_dummies(train.item, prefix = 'item')
test_item_dummies = pd.get_dummies(test.item, prefix = 'item')

train_day_dummies = pd.get_dummies(train.day, prefix = 'day')
test_day_dummies = pd.get_dummies(test.day, prefix = 'day')

In [ ]:
train_x = pd.concat([train_x, train_store_dummies, train_item_dummies, train_day_dummies], axis = 1)
test_x = pd.concat([test_x, test_store_dummies, test_item_dummies, test_day_dummies], axis = 1)

In [ ]:
# train_x = pd.concat([train_x, train_day_dummies], axis = 1)
# test_x = pd.concat([test_x, test_day_dummies], axis = 1)

In [ ]:
# ## 2-3) 불필요 데이터 삭제 ([store, item, date, day, month])

# train_x.drop(['date', 'store', 'item', 'day'], axis = 1, inplace = True)
# test_x.drop(['date', 'store', 'item', 'day'], axis = 1, inplace = True)

train_x.drop(['store', 'item', 'date', 'day', 'month'], axis = 1, inplace = True)
test_x.drop(['store', 'item', 'date', 'day', 'month'], axis = 1, inplace = True)

In [ ]:
display(train_x.head())
display(test_x.head())
display(train_y.head())

In [ ]:
## 2-4) sales 데이터의 자연로그화 (이상값 보정을 위한 방법들 중 하나로 활용 : 예측값 산출 후에는 지수e로 원위치)

train_y = np.log(train_y + 1)

sns.set_style("whitegrid")
plt.figure(figsize = (30, 5))
sns.distplot(train_y);

---

# 3. 1차 머신러닝 모델링 : 랜덤포레스트

- 기본적인 세팅은 완료되었습니다. 이제 머신러닝을 시작합니다.
- 일단 초기 머신러닝은 랜덤포레스트로 실시하였습니다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

In [ ]:
rf.fit(train_x, train_y)

In [ ]:
test_y = rf.predict(test_x)
test_y

In [ ]:
submission = pd.read_csv(submission_url)

In [ ]:
submission['sales'] = np.exp(test_y) - 1

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index = False)

**앞으로 생각해 보고 있는 방향**

1. 머신러닝 기법 변경과 하이퍼파라미터 튜닝 찾기

2. sales의 아웃라이어 특성 탐색과 처리 방향 설정

3. 변수 재조합 방향 찾기

In [ ]:
# train.groupby('item').sales.describe().sort_values(by = 'mean', ascending = False)

In [ ]:
# # Year : 연도가 올라갈수록 Sales 양이 증가함. Stores와 비교해서도 마찬가지.

# sns.set_style("whitegrid")
# plt.figure(figsize = (30, 10))
# sns.boxplot(data = train, x = 'store', y = 'sales', hue = 'year');

In [ ]:
# sns.set_style("whitegrid")
# plt.figure(figsize = (30, 10))
# sns.boxplot(data = train, x = 'store', y = 'sales', hue = 'month');

In [ ]:
# # Year : 연도가 올라갈수록 Sales 양이 증가함. Stores와 비교해서도 마찬가지.

# sns.set_style("whitegrid")
# plt.figure(figsize = (30, 10))
# sns.boxplot(data = train, x = 'store', y = 'sales', hue = 'dayname');

In [ ]:
# # 1-2) store 변수 확인
# train.store.value_counts()

In [ ]:
# sns.set_style("whitegrid")
# plt.figure(figsize = (20, 10))
# sns.boxplot(data = train, x = 'store', y = 'sales');